In [33]:
import ee
import geemap
import pandas as pd

In [34]:
Map = geemap.Map()

In [35]:
def registros(cod, m2, ha):
    diccionario = {}
    diccionario["COMUNA"] = cod
    diccionario[year + str("_m2")] = m2
    diccionario[year + str("_ha")] = ha

    return diccionario

In [36]:
df = pd.read_excel("comunas.xlsx")
comunas = df["COD_COM"].unique().tolist()

In [37]:
startDate = ['2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01']
endDate = ['2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01']

In [38]:
for j in zip(startDate,endDate):
    
    Fecha_inicial = str(j[0])
    Fecha_final = str(j[1])
    
    year = Fecha_inicial[:4]
    
    # year_text_m2 = year
    # year_text_ha = year
    
    print(year)
    puntos = []

    for i in comunas:
        studyArea = ee.FeatureCollection("users/testHector/Lim_comunas")
        studyArea = studyArea.filterMetadata("COMUNA","equals", str(i))

        dataset = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date(Fecha_inicial, Fecha_final))
        image = dataset.sum()


        mask = image.select(0).mask().rename('mask')
        area = ee.Image.pixelArea().multiply(mask).rename('area')


        #sumDictionarypolygon1 = mask.addBands(area).reduceRegion({
        sumDictionarypolygon1 = area.reduceRegion(
          reducer = ee.Reducer.sum(),
          geometry = studyArea.geometry(),
          scale = 500,
          maxPixels = 1e9
        )

        # print('sum for pol 1 mask, scale=500', sumDictionarypolygon1.values(obj))
        area_m2 = sumDictionarypolygon1.getInfo()
        
        aQuemada_m2 = float(area_m2['area'])
        aQuemada_ha = float(area_m2['area']) / 10000

        # print(aQuemada)

        diccionario = registros(i, aQuemada_m2, aQuemada_ha)
        puntos.append(diccionario.copy())
        
    data = pd.DataFrame(puntos)
    data.to_excel("ha_quemada_comuna_anual/" + str(year) + ".xlsx", index=False)

2011
2012
2013
2014
2015
